In [31]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk
import string
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [32]:
trainData = pd.read_csv(filepath_or_buffer='data/train.dat.txt', header=None, sep='\t',names=["rating","review"])[0:1]
testData =  pd.read_csv(filepath_or_buffer='data/test.dat.txt', header=None,sep='\n',names=["review"])[0:1]


In [33]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

def preProcess(data):
#     print(data)
    data["review"] = [BeautifulSoup(text).get_text() for text in data["review"] ]
    data["review"] = data["review"].map(lambda x: re.sub(r'\W+', " ", x))
    data["review"] = data["review"].map(lambda x: re.sub(r'\d+', "", x))
    data["review"] = data["review"].map(lambda x: re.sub(r'\b\w{1,3}\b', "", x))
    data["review"] = data["review"].map(lambda x: WordNetLemmatizer().lemmatize(x))
    data["review"] = data["review"].map(lambda x: PorterStemmer().stem(x))
    data["review"] = data["review"].str.lower().str.split(" ")
    stop_words = stopwords.words('english')
    data["review"] = data["review"].apply(lambda x: [item for item in x if item not in stop_words])
#   data["review"] = data["review"].apply(lambda x: remove_punctuation(x))
    return data;
    
trainData = preProcess(trainData)
print(trainData['review'])
testData = preProcess(testData)
print('pre-processing done')

0    [although, , film, bruce, willis, , always, wo...
Name: review, dtype: object
pre-processing done


In [20]:
idx = {}
tid = 0
nnz = 0
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    global idx
    global tid
    global nnz
    nrows = len(docs)
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
    return ncols,nrows

def combine_matrix(ncols, nrows, docs):
        
    # set up memory
    global nnz
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    global idx
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
    
    mat = csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)
    mat.sort_indices()
    return mat

ncols1, nrows1 = build_matrix(trainData["review"])
ncols2, nrows2 = build_matrix(testData["review"])

trainCSR = combine_matrix(ncols1, nrows1, trainData["review"])
testCSR = combine_matrix(ncols2, nrows2, testData["review"])

In [21]:
def csr_l2normalize(mat,nnz, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    nrows = mat.shape[0]
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
    return csr_matrix((val, ind, ptr), shape=(nrows, nnz), dtype=np.double)

# scale and normalize the matrix
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

In [22]:
global normTrain
global normTest

scaledTrain = csr_idf(trainCSR,copy=True)
scaledTest = csr_idf(testCSR,copy=True)

normTrain = csr_l2normalize(trainCSR, nnz)
normTest = csr_l2normalize(testCSR, nnz)

In [23]:
def classify(x, train, clstr, k=17):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(k)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]

In [24]:
try:
    outFile = open("data/final-output3.dat", 'w')
    l = [ classify(normTest[i], normTrain, trainData["rating"]) for i in range(normTest.shape[0]) ]
    for i in range(len(l)):
        if str(l[i])=="1" or str(l[i])=="+1":
            outFile.write("+1\n")
        elif str(l[i])=="-1":
            outFile.write("-1\n")
except IOError as (errno, strerror):
     print "I/O error({0}): {1}".format(errno, strerror)
finally:
    outFile.close()